In [ ]:
# Training a model 
# in less than 20k params 
# with validation accuracy 99.4%
# within 20 epochs

In [ ]:
# import all the necessary packages for training the model
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary
from tqdm import tqdm

In [ ]:
# properly used batchnorm and dropout along with 1 x 1 conv for reducing the num of channels
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conva1_1 = nn.Conv2d(32, 16, kernel_size=1)
        self.drop1 = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.convb1_1 = nn.Conv2d(32, 16, kernel_size=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(32)
        self.convc1_1 = nn.Conv2d(32, 16, kernel_size=1)
        self.conv4 = nn.Conv2d(16, 16, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.drop1(self.conv1_bn(F.relu(self.conv1(x), 2))) # 28 -> 26
        x = F.relu(self.conva1_1(x), 2) # 26 -> 26
        x = self.drop1(self.conv2_bn(F.relu(F.max_pool2d(self.conv2(x), 2)))) # 26 -> 24 -> 12
        x = F.relu(self.convb1_1(x), 2) # 12 -> 12
        x = self.drop1(self.conv3_bn(F.relu(self.conv3(x), 2))) # 12 -> 10
        x = F.relu(self.convc1_1(x), 2) # 10 -> 10
        x = self.conv4_bn(F.relu(F.max_pool2d(self.conv4(x), 2))) # 10 -> 8 -> 4
        x = x.view(-1, 256) # 4*4 * 16
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
           Dropout-3           [-1, 32, 26, 26]               0
            Conv2d-4           [-1, 16, 26, 26]             528
            Conv2d-5           [-1, 32, 24, 24]           4,640
       BatchNorm2d-6           [-1, 32, 12, 12]              64
           Dropout-7           [-1, 32, 12, 12]               0
            Conv2d-8           [-1, 16, 12, 12]             528
            Conv2d-9           [-1, 32, 10, 10]           4,640
      BatchNorm2d-10           [-1, 32, 10, 10]              64
          Dropout-11           [-1, 32, 10, 10]               0
           Conv2d-12           [-1, 16, 10, 10]             528
           Conv2d-13             [-1, 16, 8, 8]           2,320
      BatchNorm2d-14             [-1, 1

In [ ]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)
for epoch in range(1, 21):
    print("epoch", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.
epoch 1


loss=0.08696962147951126 batch_id=937: 100%|██████████| 938/938 [00:26<00:00, 36.01it/s]



Test set: Average loss: 0.0421, Accuracy: 9864/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-02.
epoch 2


loss=0.0675819143652916 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 43.73it/s]



Test set: Average loss: 0.0337, Accuracy: 9886/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-02.
epoch 3


...


Test set: Average loss: 0.0176, Accuracy: 9942/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
epoch 19


loss=0.0013028460089117289 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 42.49it/s]



Test set: Average loss: 0.0174, Accuracy: 9944/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
epoch 20


loss=0.00025359756546095014 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.83it/s]



Test set: Average loss: 0.0170, Accuracy: 9947/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
